# simulate peaks

In [27]:
from peaks_v1 import peakslocation


import numpy as np
import math
import time
import warnings
import matplotlib.pyplot as plt
import random
import cv2
import cProfile
from PIL import Image
import copy
from scipy.ndimage import gaussian_filter
from skimage.measure import block_reduce
from collections import deque

import itertools
from scipy.optimize import minimize

from kdtree import KDTree
from node import Node

from scipy import stats

from tune import tune_gs

from int_plot import int_plot_error,int_sliceviewer


GLOBAL_SEED = 42



In [28]:

sample =      np.load('../../data/ws-data/br-wsdata/data_sample_44752_5C.npy')
empty =       np.load('../../data/ws-data/br-wsdata/data_empty_44445_5C.npy')
mask_sample = np.load('../../data/ws-data/br-wsdata/mask_sample_44752_bool_5C.npy')
mask_empty =  np.load('../../data/ws-data/br-wsdata/mask_empty_44445_bool_5C.npy')
mask=np.logical_and(mask_sample,mask_empty)
# detector 14 is not counted in sample measreument (run # 44752)

sample_trim=sample[1:,1:500,:251]
empty_trim=empty[1:,1:500,:251]
mask_trim=mask[1:,1:500,:251]



In [29]:

from multiprocessing import Process,Pool

simulatedpeaksdata=np.zeros(sample_trim.shape)
xind_offset=250
yind_offset=250
zind_offset=250
xgridlen=12.5/250
ygridlen=12.5/250
zgridlen=12.5/250
gridlen=0.05
gaussian_width=0.07/gridlen
gaussian_height=1000
peakslocation_converted=np.array(peakslocation)/gridlen+np.array([xind_offset,yind_offset,zind_offset])



sp1d=[]
for ind,x in np.ndenumerate(simulatedpeaksdata):
    sp1d.append(ind)


In [30]:
#########################################
################ crash with mem error
########################################
#def generate_gaussianpeaks(data):
# 
#    for ind in data:
#      #if(mask[ind]):
#        r=np.array(ind)-peakslocation_converted
#        rmin=min(np.linalg.norm(r,axis=1))
#        simulatedpeaksdata[ind]=np.exp(-(rmin/gaussian_width)**2)*gaussian_height
#
#with Pool(processes=32) as pool:
#    pool.map(generate_gaussianpeaks,np.ndenumerate(sp1d)       )     


In [31]:

for peakloc in peakslocation_converted:
    peaklocind=[int(i) for i in peakloc]
    for xind in range(-10,11):
        for yind in range(-10,11):
            for zind in range(-10,11):
                rel_ind=(xind,yind,zind)
                r=np.array(rel_ind)+np.array(peaklocind)-peakloc

                rdis=np.linalg.norm(r)
                
                ab_ind=[int(i) for i in np.array(rel_ind)+np.array(peaklocind)]
                if (ab_ind[0]<499 and ab_ind[1]<498 and ab_ind[2]<250):
                    ind=tuple(ab_ind)
                    if (mask_trim[ind]):
                        simulatedpeaksdata[ind]=np.exp(-(rdis/gaussian_width)**2)*gaussian_height

        


In [32]:
int_sliceviewer(simulatedpeaksdata)

interactive(children=(Dropdown(description='Axis:', options=('X', 'Y', 'Z'), value='X'), IntSlider(value=0, de…

In [33]:
np.save('save_nc_simpeak.npy',simulatedpeaksdata)